In [1]:
import pkg_resources 
import pandas as pd
OutputDataSet = pd.DataFrame(sorted([(i.key, i.version) for i in pkg_resources.working_set])) 
print(OutputDataSet)

# 전체 칼럼 출력하기
pd.options.display.max_columns = None

                      0       1
0             alabaster  0.7.12
1       anaconda-client   1.7.2
2    anaconda-navigator   2.0.3
3      anaconda-project   0.9.1
4                 anyio   2.2.0
..                  ...     ...
250                yapf  0.31.0
251                zict   2.0.0
252                zipp   3.4.1
253          zope.event   4.5.0
254      zope.interface   5.3.0

[255 rows x 2 columns]


## 1.  고객 3500명에 대한 학습용 데이터를 이용하여 성별 예측 모형을 만든 후, 평가용 데이터에 적용하여 얻은 예측값(남자일 확률)을 csv 파일로 생성. 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 하이퍼 파라미터 최적화, 모형 앙상블 등이 수반

In [21]:
### 데이터 불러오기
x_train = pd.read_csv('./bigData/x_train.csv', encoding='CP949')
y_train = pd.read_csv('./bigData/y_train.csv', encoding='CP949')
x_test = pd.read_csv('./bigData/x_test.csv', encoding='CP949')

### 데이터 탐색
# print(x_train.info())
# print(y_train.info())
# print(x_test.info())

### 데이터 전처리
x_train = x_train.drop(columns = ['cust_id'])
x_test_custId = x_test['cust_id']
x_test = x_test.drop(columns = ['cust_id'])
y_train = y_train.drop(columns = ['cust_id'])

# print(x_train.isnull().sum())
# print(x_test.isnull().sum())
# print(pd.concat([x_train, y_train], axis=1).corr())

x_train['환불금액'] = x_train['환불금액'].fillna(x_train['환불금액'].mean())
x_test['환불금액'] = x_test['환불금액'].fillna(x_test['환불금액'].mean())

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
x_train['주구매상품'] = encoder.fit_transform(x_train['주구매상품'])
x_train['주구매지점'] = encoder.fit_transform(x_train['주구매지점'])
x_test['주구매상품'] = encoder.fit_transform(x_test['주구매상품'])
x_test['주구매지점'] = encoder.fit_transform(x_test['주구매지점'])
# print(x_train.info())
# print(y_train.info())

# print(x_train[['총구매액', '최대구매액', '환불금액']].corr())
x_train = x_train.drop(columns = ['최대구매액'])
x_test = x_test.drop(columns = ['최대구매액'])

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

### 데이터 학습
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_train_pred = pd.DataFrame(model.predict(x_train))
y_test_pred = pd.DataFrame(model.predict(x_test))

### 모델 평가
from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_train, y_train_pred))

### 결과 저장
result = pd.concat([x_test_custId, y_test_pred], axis=1)
result.columns = ['cust_id', 'gender']
result.to_csv('./bigData/result1.csv', index = False)
# print(pd.read_csv('./bigData/result1.csv'))

<ipython-input-21-3497745f5106>:45: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


0.9995421245421245
      cust_id  gender
0        3500       1
1        3501       0
2        3502       0
3        3503       0
4        3504       1
...       ...     ...
2477     5977       1
2478     5978       1
2479     5979       1
2480     5980       1
2481     5981       1

[2482 rows x 2 columns]


## 2. 고객 891명에 대한 학습용 데이터를 이용하여 생존 여부를 예측하는 모형을 만든다. 이후 예측 모형을 평가용 데이터에 적용하여 418명 승객의 생존 여부 예측값을 CSV 파일로 생성

In [55]:
### 데이터 불러오기
x_train = pd.read_csv('./bigData/titanic_x_train.csv', encoding='CP949')
x_test = pd.read_csv('./bigData/titanic_x_test.csv', encoding='CP949')
y_train = pd.read_csv('./bigData/titanic_y_train.csv', encoding='CP949')
y_train.columns = ['PassengerId', 'Survived']

# print(x_train.head())
# print(x_test.head())
# print(y_train.head())
# print(x_train.info())
# print(x_test.info())
# print(y_train.info())
# print(x_train.isnull().sum())

x_train = x_train.drop(columns = ['PassengerId', '승객이름', '티켓번호', '객실번호', '나이'])
x_test_passengerId = x_test['PassengerId']
x_test = x_test.drop(columns = ['PassengerId', '승객이름', '티켓번호', '객실번호', '나이'])
y_train = y_train.drop(columns = ['PassengerId'])

# print(pd.concat([x_train, y_train], axis=1).corr()['Survived'])
# print(x_train['선착장'].mode())
x_train['선착장'] = x_train['선착장'].fillna('S')
x_test['운임요금'] = x_test['운임요금'].fillna(x_test['운임요금'].mean())
# print(x_train.isnull().sum())
# print(x_test.isnull().sum())

x_train['성별'] = x_train['성별'].replace('male', 0).replace('female', 1)
x_test['성별'] = x_test['성별'].replace('male', 0).replace('female', 1)
x_train['가족수'] = x_train['형제자매배우자수'] + x_train['부모자식수']
x_test['가족수'] = x_test['형제자매배우자수'] + x_test['부모자식수']
x_train = x_train.drop(columns = ['형제자매배우자수', '부모자식수'])
x_test = x_test.drop(columns = ['형제자매배우자수', '부모자식수'])

new_train_선착장 = pd.get_dummies(x_train['선착장'], drop_first = True)
new_train_선착장.columns = ['선착장Q', '선착장S']
x_train = pd.concat([x_train.drop(columns = ['선착장']), new_train_선착장], axis=1)
new_test_선착장 = pd.get_dummies(x_test['선착장'], drop_first = True)
new_test_선착장.columns = ['선착장Q', '선착장S']
x_test = pd.concat([x_test.drop(columns = ['선착장']), new_test_선착장], axis=1)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# x_train = pd.DataFrame(scaler.fit_transform(x_train), columns = x_train.columns)
# x_test = pd.DataFrame(scaler.fit_transform(x_test), columns = x_test.columns)

# print(x_train.describe())

### 데이터 학습 및 모델 평가
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(x_train, y_train)
y_train_pred = pd.DataFrame(model.predict(x_train))
y_test_pred = pd.DataFrame(model.predict(x_test))

# from sklearn.tree import DecisionTreeClassifier
# model = DecisionTreeClassifier()
# model.fit(x_train, y_train)
# y_train_pred = model.predict(x_train)
# y_test_pred = model.predict(x_test)

from sklearn.metrics import roc_auc_score
# print(roc_auc_score(y_train, y_train_pred))

### 결과 저장
result = pd.concat([x_test_passengerId, y_test_pred], axis=1)
result.columns = ['PassengerId', 'Survived']
result.to_csv('./bigData/result2.csv', index = False)
print(pd.read_csv('./bigData/result2.csv'))

     PassengerId  Survived
0            892         0
1            893         1
2            894         0
3            895         0
4            896         1
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]


<ipython-input-55-d1754a41a4f1>:51: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
